## **Lightcurve and transmission spectra generation using Prometheus**

#### Developed by *Rishabh Garg*

In [1]:
# Imports

from datetime import datetime
from typing import Any, Dict, List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator, ScalarFormatter
from scipy.optimize import curve_fit

import Prometheus.pythonScripts.celestialBodies as bodies
import Prometheus.pythonScripts.constants as const
import Prometheus.pythonScripts.gasProperties as gasprop
import Prometheus.pythonScripts.geometryHandler as geom

from exomoon_orbits_tool import TimingModel

import ldtk

Cannot import astroquery.simbad


/home/eclipse/exoplanets/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

In [2]:
# Parameters for different scenarios

def get_exomoon_params(
    planet_name: str,
    element_name: str,
    num_particles: float,
    sigma_v: float,
    moon_radius_io: float,
    a_moon: float,
    # Exomoon's phase (radians) when planet is at mid-transit (phase 0)
    starting_orbphase_moon_rad: float,
    moon_q: float,
    light_curve: bool = False,
    # observed_planet_phases: These are the orbital phases OF THE PLANET for which to run the simulation
    observed_planet_phases: Optional[np.ndarray] = None,
    # observed_wavelength_transmissions: Wavelengths (Angstroms) from an observed spectrum file for grid setup
    observed_wavelength_transmissions: Optional[np.ndarray] = None
) -> Dict[str, Any]:
    """
    Sets up parameters for a generic planet with an exomoon scenario.
    """
    planet_obj = bodies.AvailablePlanets().findPlanet(planet_name)
    if planet_obj is None:
        raise ValueError(
            f"{planet_name} not found in AvailablePlanets.")

    hill_radius = planet_obj.a * \
        (planet_obj.M / (3 * planet_obj.hostStar.M))**(1/3)
    if a_moon == 0:
        a_moon = 0.49 * hill_radius  # 0.49 * Hill radius - Oza et. al 2019, ApJ

    params: Dict[str, Dict[str, Any]] = {
        "Fundamentals": {
            "ExomoonSource": True,
            "DopplerPlanetRotation": False,
            "CLV_variations": True,
            "RM_effect": False,
            "DopplerOrbitalMotion": True,
        },
        "Architecture": {
            "planetName": planet_name,
            "R_moon": moon_radius_io * const.R_Io,
            "a_moon": a_moon,
            # This is the exomoon's phase # 0: Observer (Earth) -----> Planet Center -----> Moon -----> Star Center
            "starting_orbphase_moon": starting_orbphase_moon_rad,
        },
        "Scenarios": {
            "exomoon": {"q_moon": moon_q}
        },
        "Species": {"exomoon": {element_name: {"Nparticles": num_particles, "sigma_v": sigma_v}}},
        "Grids": {
            "x_midpoint": planet_obj.a,
            "x_border": 5.0 * hill_radius,
            "x_steps": 30,
            "phi_steps": 40,  # Azimuthal steps on stellar disk
            "rho_steps": 30,  # Radial steps on stellar disk
            "upper_rho": planet_obj.hostStar.R,  # Max radial extent on stellar disk
            # These orbphase settings are for the PLANET's orbit
            "orbphase_border": 0,  # Default for single spectrum (mid-transit)
            "orbphase_steps": 1,  # Default for single spectrum
        },
    }

    # If in light curve mode, adjust PLANET'S orbphase parameters based on observed_planet_phases
    if light_curve:
        if observed_planet_phases is not None and len(observed_planet_phases) > 0:
            # observed_planet_phases are fractions of an orbit (e.g., -0.05 to 0.05 for around transit)
            min_obs_planet_phase_frac = np.min(observed_planet_phases)
            max_obs_planet_phase_frac = np.max(observed_planet_phases)
            # orbphase_border needs to be in RADIANS and symmetric around 0 for Prometheus
            # It defines the range [-orbphase_border, +orbphase_border]
            planet_phase_border_fractional = max(
                abs(min_obs_planet_phase_frac), abs(max_obs_planet_phase_frac))
            # Add a small margin to ensure coverage
            params["Grids"]["orbphase_border"] = (
                planet_phase_border_fractional + 0.005) * 2.0 * np.pi
        else:
            # Default phase coverage for a light curve if no observed phases are given
            params["Grids"]["orbphase_border"] = 0.065 * \
                2.0 * np.pi  # e.g., +/- 0.065 of an orbit

        # Number of planet orbital phase points
        params["Grids"]["orbphase_steps"] = 201

    # Set up element-specific wavelength grids and CLV
    if element_name == "NaI":
        grids_Na = {
            "lower_w": 5.888e-05,  # cm (5888 Angstroms) - Vacuum
            "upper_w": 5.900e-05,  # cm (5900 Angstroms) - Vacuum
            "resolutionLow": 5e-09,  # cm
            # cm (This is 0.75 Angstrom, consistent with filter)
            "widthHighRes": 0.75e-8,
            "resolutionHigh": 2e-10,  # cm
        }
        params["Grids"].update(grids_Na)
        try:
            # LDTk filters are usually defined with wavelengths in Angstroms
            # The wavelengths 5888-5900 A cover the Na D lines.
            filters_Na = [ldtk.BoxcarFilter("NaD_filter", 5888, 5900)]
            sc_Na = ldtk.LDPSetCreator(
                teff=(planet_obj.hostStar.T_eff, planet_obj.hostStar.dT_eff),
                logg=(planet_obj.hostStar.log_g, planet_obj.hostStar.dlog_g),
                z=(planet_obj.hostStar.Z, planet_obj.hostStar.dZ),
                filters=filters_Na,
            )
            ps_Na = sc_Na.create_profiles()
            u1_Na, u2_Na = ps_Na.coeffs_qd(do_mc=True)
            params["Architecture"]["CLV_u1"] = u1_Na[0][0]
            params["Architecture"]["CLV_u2"] = u2_Na[0][0]
        except Exception as e:
            print(f"Error creating LDTk profiles for NaI: {e}")
            # Fallback to default CLV if LDTk fails
            params["Architecture"]["CLV_u1"] = 0
            params["Architecture"]["CLV_u2"] = 0

    elif element_name == "KI":
        grids_K = {
            "lower_w": 7.660e-05,  # cm (7660 Angstroms) - Vacuum
            "upper_w": 7.705e-05,  # cm (7705 Angstroms) - Vacuum
            "resolutionLow": 5e-09,  # cm
            "widthHighRes": 1.0e-08,  # cm (1.0 Angstrom)
            "resolutionHigh": 2e-10,  # cm
        }
        params["Grids"].update(grids_K)
        filters_K = [ldtk.BoxcarFilter("KD_filter", 7660, 7705)]
        sc_K = ldtk.LDPSetCreator(
            teff=(planet_obj.hostStar.T_eff, planet_obj.hostStar.dT_eff),
            logg=(planet_obj.hostStar.log_g, planet_obj.hostStar.dlog_g),
            z=(planet_obj.hostStar.Z, planet_obj.hostStar.dZ),
            filters=filters_K,
        )
        ps_K = sc_K.create_profiles()
        u1_K, u2_K = ps_K.coeffs_qd(do_mc=True)
        params["Architecture"]["CLV_u1"] = u1_K[0][0]
        params["Architecture"]["CLV_u2"] = u2_K[0][0]
    else:
        print(
            f"Warning: Element {element_name} specific grid not fully set. Using generic wide grid.")
        params["Grids"].update({
            "lower_w": 0.5e-4, "upper_w": 1.0e-4, "resolutionLow": 1e-8,
            "widthHighRes": 1e-8, "resolutionHigh": 1e-9
        })
        # Default CLV if not NaI or KI, or if LDTk fails for some reason
        params["Architecture"]["CLV_u1"] = 0.3
        params["Architecture"]["CLV_u2"] = 0.1

    # If generating a transmission spectrum (not light curve) AND observed wavelengths are provided
    if not light_curve and observed_wavelength_transmissions is not None:
        print("Transmission spectrum mode: Overwriting wavelength grid from observed data file (if provided).")
        # Assuming observed_wavelength_transmissions are in Angstroms
        obs_w_cm = observed_wavelength_transmissions * 1e-8
        min_wavelength_cm = np.min(obs_w_cm)
        max_wavelength_cm = np.max(obs_w_cm)
        # Add a small padding to ensure the observed range is fully covered
        padding = (max_wavelength_cm - min_wavelength_cm) * 0.01
        params["Grids"]["lower_w"] = min_wavelength_cm - padding
        params["Grids"]["upper_w"] = max_wavelength_cm + padding
        # Resolutions might need adjustment if the observed range is very different
        print(f"New wavelength grid from observed data (cm): "
              f"{params['Grids']['lower_w']:.3e} to {params['Grids']['upper_w']:.3e}")
        # Forcing a generic high resolution if using observed data range
        params["Grids"]["resolutionLow"] = (
            params["Grids"]["upper_w"] - params["Grids"]["lower_w"]) / 1000
        params["Grids"]["widthHighRes"] = params["Grids"]["resolutionLow"] * 10
        params["Grids"]["resolutionHigh"] = params["Grids"]["resolutionLow"] / 10

    return params


def get_hydrostatic_params(
    planet_name: str,
    element_name: str,
    mixing_ratio: float,
    temperature: float,
    reference_pressure_bar: float,
    mean_molecular_weight_amu: float,
    light_curve: bool = False,
) -> Dict[str, Any]:
    """
    Sets up parameters for a planet with a hydrostatic atmosphere scenario.

    This function is designed to be flexible for any planet in the database.

    Args:
        planet_name (str): Name of the exoplanet (e.g., "HD189733b").
        element_name (str): Name of the absorbing species (e.g., "NaI", "KI").
        mixing_ratio (float): The volume mixing ratio of the species (e.g., 1.7e-6 for solar).
        temperature (float): Isothermal temperature of the atmosphere in Kelvin.
        reference_pressure_bar (float): The reference pressure at the planet's radius (R_p) in bars.
        mean_molecular_weight_amu (float): Mean molecular weight of the background gas in AMU (e.g., 2.3 for H2/He).
        light_curve (bool): Set to True to generate a light curve, False for a single transmission spectrum.
        observed_planet_phases (Optional[np.ndarray]): For light curves, an array of observed planet phases
            to set the simulation grid.
        observed_wavelength_transmissions (Optional[np.ndarray]): For transmission spectra, an array of
            observed wavelengths (in Angstroms) to set the wavelength grid.

    Returns:
        Dict[str, Any]: A parameter dictionary ready for use with Prometheus.
    """
    planet_obj = bodies.AvailablePlanets().findPlanet(planet_name)
    if planet_obj is None:
        raise ValueError(f"Planet '{planet_name}' not found in the database.")

    # Convert user-friendly units to cgs for Prometheus
    pressure_cgs = reference_pressure_bar * 1e6  # 1 bar = 1e6 dyne/cm^2
    mu_cgs = mean_molecular_weight_amu * const.amu

    params: Dict[str, Dict[str, Any]] = {
        "Fundamentals": {
            "ExomoonSource": False,  # This is the key difference from the exomoon model
            "DopplerPlanetRotation": False,
            "CLV_variations": True,
            "RM_effect": False,
            "DopplerOrbitalMotion": True,
        },
        "Architecture": {
            "planetName": planet_name,
            # CLV parameters will be added below
        },
        "Scenarios": {
            "hydrostatic": {
                "T": temperature,
                "P_0": pressure_cgs,
                "mu": mu_cgs
            }
        },
        "Species": {
            "hydrostatic": {
                element_name: {"chi": mixing_ratio}
            }
        },
        "Grids": {
            "x_midpoint": planet_obj.a,
            "x_border": 5.0 * planet_obj.R,  # Integrate over a chord of +/- 5 planetary radii
            "x_steps": 200,
            "phi_steps": 60,
            "rho_steps": 40,
            "upper_rho": planet_obj.hostStar.R,
            "orbphase_border": 0,  # Default for single spectrum
            "orbphase_steps": 1,   # Default for single spectrum
        },
    }

    # --- Light Curve Grid Setup ---
    if light_curve:
        params["Grids"]["orbphase_border"] = 0.05 * 2.0 * np.pi
        params["Grids"]["orbphase_steps"] = 201

    # --- Wavelength Grid and CLV for Sodium/Potassium ---
    if element_name == "NaI":
        params["Grids"].update({
            "lower_w": 5.888e-05, "upper_w": 5.900e-05, "resolutionLow": 5e-09,
            "widthHighRes": 0.75e-8, "resolutionHigh": 2e-10,
        })
        filters = [ldtk.BoxcarFilter("NaD_filter", 5888, 5900)]
    elif element_name == "KI":
        params["Grids"].update({
            "lower_w": 7.660e-05, "upper_w": 7.705e-05, "resolutionLow": 5e-09,
            "widthHighRes": 1.0e-8, "resolutionHigh": 2e-10,
        })
        filters = [ldtk.BoxcarFilter("KD_filter", 7660, 7705)]
    else:
        print(
            f"Warning: Wavelength grid not pre-defined for {element_name}. Using generic wide grid.")
        params["Grids"].update({
            "lower_w": 0.5e-4, "upper_w": 1.0e-4, "resolutionLow": 1e-8,
            "widthHighRes": 1e-8, "resolutionHigh": 1e-9
        })
        # Generic filter for LDTk
        filters = [ldtk.BoxcarFilter("Generic_filter", 5000, 10000)]

    # Calculate and add CLV parameters
    try:
        sc = ldtk.LDPSetCreator(
            teff=(planet_obj.hostStar.T_eff, planet_obj.hostStar.dT_eff),
            logg=(planet_obj.hostStar.log_g, planet_obj.hostStar.dlog_g),
            z=(planet_obj.hostStar.Z, planet_obj.hostStar.dZ),
            filters=filters,
        )
        ps = sc.create_profiles()
        u1, u2 = ps.coeffs_qd(do_mc=True)
        params["Architecture"]["CLV_u1"] = u1[0][0]
        params["Architecture"]["CLV_u2"] = u2[0][0]
    except Exception as e:
        print(
            f"LDTk failed to generate CLV coefficients: {e}. Using default values.")
        params["Architecture"]["CLV_u1"] = 0.3
        params["Architecture"]["CLV_u2"] = 0.1
    return params


def get_plume_params(planet_name: str,
                     element_name: str,
                     num_particles: float,
                     sigma_v: float,
                     plume_radius_km: float,
                     # Exomoon's phase (radians) when planet is at mid-transit (phase 0)
                     plume_longitude: float,
                     moon_q: float,
                     light_curve: bool = False,
                     # observed_planet_phases: These are the orbital phases OF THE PLANET for which to run the simulation
                     observed_planet_phases: Optional[np.ndarray] = None,
                     # observed_wavelength_transmissions: Wavelengths (Angstroms) from an observed spectrum file for grid setup
                     observed_wavelength_transmissions: Optional[np.ndarray] = None
                     ) -> Dict[str, Any]:
    plume_radius = plume_radius_km * 1e5  # Convert km to cm
    planet_obj = bodies.AvailablePlanets().findPlanet(planet_name)
    params = get_exomoon_params(
        planet_name, element_name, num_particles, sigma_v, plume_radius,
        planet_obj.R, plume_longitude, moon_q, light_curve,
        observed_planet_phases, observed_wavelength_transmissions
    )
    return params

In [3]:
def load_observed(filepath: str) -> Tuple[Optional[np.ndarray], Optional[np.ndarray], Optional[np.ndarray]]:
    """Loads observed data from a 2 or 3 column text file."""
    try:
        data = np.loadtxt(filepath)
        if data.ndim == 1:
            if len(data) == 3:
                return np.array([data[0]]), np.array([data[1]]), np.array([data[2]])
            elif len(data) == 2:
                return np.array([data[0]]), np.array([data[1]]), np.array([0.0])
            else:
                raise ValueError(
                    f"Single row in data file {filepath} has incorrect number of columns.")

        if data.shape[1] == 3:
            return data[:, 0], data[:, 1], data[:, 2]
        elif data.shape[1] == 2:
            print(
                f"Warning: Data file {filepath} has 2 columns. Assuming zero error.")
            return data[:, 0], data[:, 1], np.zeros_like(data[:, 0])
        else:
            raise ValueError(f"Data file {filepath} must have 2 or 3 columns.")
    except Exception as e:
        print(f"Error loading observed data from {filepath}: {e}")
        return None, None, None

# Get initial parameters

Moon Exospheres at Q=3.34 from Oza et. al 2019 ApJ Equation 16 

In [4]:
PLANET_NAME = input("Enter the planet name (e.g., WASP-49b): ").strip()
ELEMENT_NAME = input("Enter the species (e.g. NaI, KI): ").strip()
NUM_PARTICLES = float(input(f"Enter number of {ELEMENT_NAME} particles: "))
SIGMA_V = float(input(f"Enter {ELEMENT_NAME} sigma_v in cm/s: "))
PLUME_RADIUS = float(input(f"Enter plume radius in km (e.g., 100): ")) * 1e5  # Convert km to cm
MOON_EXOSPHERE_Q = 3.34

# Light Curve Generation

In [5]:
obs_phases, obs_flux, obs_errors = None, None, None
obs_wavelength, obs_transmission = None, None

OBSERVED_DATA_FILEPATH = input(
    "Enter path to observed light curve data: ").strip()
FILTER_BANDWIDTH_ANGSTROM = 0.75
FILTER_BANDWIDTH_CM = FILTER_BANDWIDTH_ANGSTROM * 1e-8
obs_phases, obs_flux, obs_errors = load_observed(
    OBSERVED_DATA_FILEPATH)
if obs_phases is None:
    print("Could not load observed light curve data. Continuing without it.")

Error loading observed data from :  not found.
Could not load observed light curve data. Continuing without it.


In [6]:
# Helper functions for determining exomoon initial orbital phase
# Only used if light curve data file is provided
def gaussian(x, amplitude, mean, stddev):
    """
    A standard Gaussian function for fitting.
    'mean' will represent the peak of the transient event.
    """
    return amplitude * np.exp(-((x - mean) / stddev)**2 / 2.0)

def convert_phase_to_minutes(phase, orbital_period_days):
    """Converts orbital phase values to time from mid-transit in minutes."""
    orbital_period_minutes = orbital_period_days * 24 * 60
    return phase * orbital_period_minutes

In [7]:
if obs_phases is not None and len(obs_phases) > 0:
    PLANET_ORBITAL_PERIOD_DAYS = bodies.AvailablePlanets().findPlanet(
        PLANET_NAME).orbitalPeriod
    time_from_mid_transit_min = convert_phase_to_minutes(obs_phases, PLANET_ORBITAL_PERIOD_DAYS)
    obs_flux_inv = -obs_flux
    print("\nData Preview:")
    print(f"Time from mid-transit (min): ~{time_from_mid_transit_min[0]:.2f} to ~{time_from_mid_transit_min[-1]:.2f}")
    print(f"Observed Absorption Depth: ~{obs_flux_inv[0]:.4f} to ~{obs_flux_inv[-1]:.4f}")
    # amplitude: should be positive since we inverted the flux
    # mean: the estimated time of the peak in minutes
    # stddev: the estimated width of the event in minutes
    initial_guesses = [np.max(obs_flux_inv), -30.0, 15.0]
    try:
        popt, pcov = curve_fit(gaussian, time_from_mid_transit_min, obs_flux_inv, p0=initial_guesses, sigma=obs_errors, maxfev=5000)

        # Extract the best-fit parameters
        fit_amplitude, fit_mean, fit_stddev = popt
        
        # The 'mean' of the Gaussian is the time of peak absorption
        T_peak_minutes = fit_mean
        
        # Calculate the duration (Full Width at Half Maximum)
        fwhm_minutes = 2.355 * abs(fit_stddev)

        print("\n--- Fit Results ---")
        print(f"Fitted Peak Absorption Time (T_peak): {T_peak_minutes:.2f} minutes from planet's mid-transit.")
        print(f"Fitted Event Duration (FWHM): {fwhm_minutes:.2f} minutes.")
        print(f"Fitted Amplitude: {fit_amplitude:.4f}")
        
        fit_successful = True
        
    except RuntimeError:
        print("\nCurve fitting failed. Could not find the peak. Check initial guesses or data quality.")
        fit_successful = False
else:
    fit_successful = False
    

In [8]:
if fit_successful:
    # Generate a smooth curve for the plot using the fit parameters
    time_fit_smooth = np.linspace(
        time_from_mid_transit_min.min(), time_from_mid_transit_min.max(), 500)
    absorption_fit_smooth = gaussian(time_fit_smooth, *popt)

    # Create the plot
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot the observed data
    ax.errorbar(time_from_mid_transit_min, obs_flux_inv, yerr=obs_errors,
                fmt='o', color='royalblue', ecolor='lightgray', capsize=4,
                label='Observed Data', zorder=5)

    # Plot the Gaussian fit
    ax.plot(time_fit_smooth, absorption_fit_smooth, color='crimson', linewidth=3,
            label='Gaussian Fit of Transient Event', zorder=10)

    # Add vertical lines for reference
    ax.axvline(0, color='black', linestyle='--',
               linewidth=2, label="Planet Mid-Transit (T=0)")
    ax.axvline(T_peak_minutes, color='darkorange', linestyle=':', linewidth=3,
               label=f'Peak Absorption Time = {T_peak_minutes:.2f} min')

    # --- Formatting and Labels ---
    ax.set_title(
        "Determining Exomoon Transit Time from Light Curve", fontsize=18, pad=20)
    ax.set_xlabel("Time from Planet Mid-Transit (minutes)", fontsize=14)
    ax.set_ylabel("Absorption Depth (-Relative Flux)", fontsize=14)
    ax.legend(fontsize=12, loc='upper left')
    ax.tick_params(axis='both', which='major', labelsize=12)

    # Invert y-axis so absorption dip points down, which is more intuitive
    ax.invert_yaxis()

    plt.tight_layout()
    plt.show()

In [9]:
if fit_successful:
    if abs(T_peak_minutes) < fwhm_minutes / 2.0:
        print("\nExomoon is likely transiting the planet during the observed event. 0.0 radians will be used as the initial prediction phase.")
        plume_longitude = 0.0
    elif abs(T_peak_minutes) > fwhm_minutes:
        if T_peak_minutes < 0:
            print("\nExomoon is is likely trailing the planet during the observed event. Phase will be set to 0.75% of the orbit (1.5 radians).")
            plume_longitude = 1.5
        else:
            print("\nExomoon is likely leading the planet during the observed event. Phase will be set to 0.25% of the orbit (0.5 radians).")
            plume_longitude = 0.5
    else:
        print("Something strange is going on! Likely the exomoon is transiting the planet, but the event is not symmetric. Using 0.0 radians as the initial prediction phase.")
        plume_longitude = 0.0
else:
    print("\nExomoon transit time could not be determined from the light curve. Using 0.0 radians as the initial prediction phase.")
    plume_longitude = 0.0


Exomoon transit time could not be determined from the light curve. Using 0.0 radians as the initial prediction phase.


In [10]:
fix_date = input("Enter the date and time of the observation (YYYY-MM-DD HH:MM:SS): ").strip()
try:
    observation_date = datetime.strptime(fix_date, "%Y-%m-%d %H:%M:%S")
except ValueError:
    print("Invalid date format. Please use YYYY-MM-DD HH:MM:SS.")

# Add input for other dates
try:
    tm = TimingModel((fix_date, plume_longitude / (2 * np.pi)))
    tm.search_orbitsolution(error_threshold=0.1)

    simulated_period = 24 * 60 * float(input("Enter the simulated orbital period of the exomoon in days: ").strip())
    phase = (T_peak_minutes / simulated_period) % 1.0
    print(f"Exomoon initial orbital phase: {phase:.4f} (in fractions of an orbit)")
except Exception as e:
    print(f"Error during timing model search: {e}")
    phase = 0.0

Invalid date format. Please use YYYY-MM-DD HH:MM:SS.
Error during timing model search: Invalid input date-string encountered. Please enter as 'YYYY-MM-DD HH:MM:SS


In [11]:
plume_longitude = 0
params = get_plume_params(
    PLANET_NAME, ELEMENT_NAME, NUM_PARTICLES, SIGMA_V, PLUME_RADIUS, plume_longitude, MOON_EXOSPHERE_Q,
    light_curve=True, observed_planet_phases=obs_phases,
    observed_wavelength_transmissions=obs_wavelength
)

fundamentalsDict = params["Fundamentals"]
scenarioDict = params["Scenarios"]
architectureDict = params["Architecture"]
speciesDict = params["Species"]
gridsDict = params["Grids"]

planet = bodies.AvailablePlanets().findPlanet(
    architectureDict["planetName"])
if fundamentalsDict["CLV_variations"]:
    planet.hostStar.addCLVparameters(
        architectureDict.get("CLV_u1", 0.0), architectureDict.get("CLV_u2", 0.0))

wavelengthGrid = gasprop.WavelengthGrid(
    gridsDict["lower_w"], gridsDict["upper_w"], gridsDict["widthHighRes"],
    gridsDict["resolutionLow"], gridsDict["resolutionHigh"])

spatialGrid = geom.Grid(
    gridsDict["x_midpoint"], gridsDict["x_border"], int(
        gridsDict["x_steps"]),
    gridsDict["upper_rho"], int(
        gridsDict["rho_steps"]), int(gridsDict["phi_steps"]),
    gridsDict["orbphase_border"], int(gridsDict["orbphase_steps"]))

Error creating LDTk profiles for NaI: index 510 is out of bounds for axis 0 with size 510


In [12]:
scenarioList = []
if "exomoon" in scenarioDict:
    moon = bodies.Moon(
        architectureDict["starting_orbphase_moon"],
        architectureDict["R_moon"], architectureDict["a_moon"], planet)
    species_name = list(speciesDict["exomoon"].keys())[0]
    num_p = speciesDict["exomoon"][species_name]["Nparticles"]
    scenarioList.append(gasprop.MoonExosphere(
        num_p, scenarioDict["exomoon"]["q_moon"], moon))

In [13]:
# Check to see if the sigma_v is correctly set and that the species are available
for idx, scenario_key in enumerate(scenarioDict.keys()):
    for species_key, absorberDict in speciesDict[scenario_key].items():
        if species_key in const.AvailableSpecies().listSpeciesNames():
            if "sigma_v" in absorberDict:
                scenarioList[idx].addConstituent(
                    species_key, absorberDict["sigma_v"])
                scenarioList[idx].constituents[-1].addLookupFunctionToConstituent(
                    wavelengthGrid)
            else:
                raise ValueError(
                    f"Missing 'sigma_v' for atomic species {species_key}")
        else:  # Molecular
            raise NotImplementedError(
                "Molecular species not fully supported in this script version.")

In [14]:
atmos = gasprop.Atmosphere(
    scenarioList, fundamentalsDict["DopplerOrbitalMotion"])
main_transit_model = gasprop.Transit(atmos, wavelengthGrid, spatialGrid)
main_transit_model.addWavelength()

print("\nRunning PROMETHEUS simulation...")
start_time = datetime.now()
R_values_all_wavelengths = main_transit_model.sumOverChords()
end_time = datetime.now()
print(
    f"PROMETHEUS simulation finished. Elapsed time: {end_time - start_time}")


Running PROMETHEUS simulation...
PROMETHEUS simulation finished. Elapsed time: 0:00:51.223333


In [15]:
if R_values_all_wavelengths.ndim == 2 and R_values_all_wavelengths.shape[0] > 0:
    orbital_phases_rad = spatialGrid.constructOrbphaseAxis()
    orbital_phases_plot = orbital_phases_rad / (2.0 * np.pi)

    light_curve_sim = []
    v_los_planet = np.zeros_like(orbital_phases_rad)
    if scenarioList and hasattr(scenarioList[0], 'moon'):
        moon_obj = scenarioList[0].moon
        v_los_planet = np.array([moon_obj.getLOSvelocity(ph)
                               for ph in orbital_phases_rad])

    shifts = const.calculateDopplerShift(v_los_planet)
    NaD1_cm, NaD2_cm = 5.897558147e-5, 5.891583253e-5

    for i, R_spectrum in enumerate(R_values_all_wavelengths):
        shift_factor = shifts[i]
        shifted_D1 = NaD1_cm / shift_factor
        shifted_D2 = NaD2_cm / shift_factor
        sel1 = (main_transit_model.wavelength >= shifted_D1 - FILTER_BANDWIDTH_CM / 2) & \
               (main_transit_model.wavelength <=
                shifted_D1 + FILTER_BANDWIDTH_CM / 2)
        sel2 = (main_transit_model.wavelength >= shifted_D2 - FILTER_BANDWIDTH_CM / 2) & \
               (main_transit_model.wavelength <=
                shifted_D2 + FILTER_BANDWIDTH_CM / 2)
        if np.any(sel1 | sel2):
            light_curve_sim.append(np.mean(R_spectrum[sel1]))
        else:
            light_curve_sim.append(1.0)
else:
    print("Simulation did not produce a valid light curve array.")

In [16]:
atmos_temp_hydro = float(input("Enter the isothermal temperature of the hydrostatic atmosphere in Kelvin: ").strip())
ref_pressure_hydro = 1.0
mix_ratio_hydro = 1.7e-6
mean_mu_hydro = 2.3

In [17]:
hydrostatic_params = get_hydrostatic_params(
    planet_name=PLANET_NAME,
    element_name=ELEMENT_NAME,
    mixing_ratio=mix_ratio_hydro,
    temperature=atmos_temp_hydro,
    reference_pressure_bar=ref_pressure_hydro,
    mean_molecular_weight_amu=mean_mu_hydro,
    light_curve=True
)

fundamentalsDict = hydrostatic_params["Fundamentals"]
scenarioDict = hydrostatic_params["Scenarios"]
architectureDict = hydrostatic_params["Architecture"]
speciesDict = hydrostatic_params["Species"]
gridsDict = hydrostatic_params["Grids"]

planet = bodies.AvailablePlanets().findPlanet(
    architectureDict["planetName"])
if fundamentalsDict["CLV_variations"]:
    planet.hostStar.addCLVparameters(
        architectureDict.get("CLV_u1", 0.0), architectureDict.get("CLV_u2", 0.0))

wavelengthGrid = gasprop.WavelengthGrid(
    gridsDict["lower_w"], gridsDict["upper_w"], gridsDict["widthHighRes"],
    gridsDict["resolutionLow"], gridsDict["resolutionHigh"])

spatialGrid = geom.Grid(
    gridsDict["x_midpoint"], gridsDict["x_border"], int(
        gridsDict["x_steps"]),
    gridsDict["upper_rho"], int(
        gridsDict["rho_steps"]), int(gridsDict["phi_steps"]),
    gridsDict["orbphase_border"], int(gridsDict["orbphase_steps"]))

LDTk failed to generate CLV coefficients: index 510 is out of bounds for axis 0 with size 510. Using default values.


In [18]:
scenario_model_hydro = gasprop.HydrostaticAtmosphere(
    hydrostatic_params["Scenarios"]["hydrostatic"]["T"],
    hydrostatic_params["Scenarios"]["hydrostatic"]["P_0"],
    hydrostatic_params["Scenarios"]["hydrostatic"]["mu"],
    planet
)
scenario_model_hydro.addConstituent("NaI", hydrostatic_params["Species"]["hydrostatic"]["NaI"]["chi"])
scenario_model_hydro.constituents[-1].addLookupFunctionToConstituent(wavelengthGrid)

In [19]:
print("\n--- Running PROMETHEUS for Hydrostatic Atmosphere... ---")
atmos_hydro = gasprop.Atmosphere([scenario_model_hydro], hydrostatic_params["Fundamentals"]["DopplerOrbitalMotion"])
transit_model_hydro = gasprop.Transit(atmos_hydro, wavelengthGrid, spatialGrid)
transit_model_hydro.addWavelength()
R_values_hydro = transit_model_hydro.sumOverChords()
print("Hydrostatic simulation finished.")
R_values_hydro.shape


--- Running PROMETHEUS for Hydrostatic Atmosphere... ---
Hydrostatic simulation finished.


(201, 99)

In [ ]:
if R_values_hydro.ndim == 2 and R_values_hydro.shape[0] > 0:
    orbital_phases_rad_hydro = spatialGrid.constructOrbphaseAxis()
    orbital_phases_plot_hydro = orbital_phases_rad_hydro / (2.0 * np.pi)

    light_curve_sim_hydro = []
    v_los_planet_hydro = np.zeros_like(orbital_phases_rad_hydro)
    # For hydrostatic, there is no moon, so just use zeros or the planet's velocity if needed

    shifts_hydro = const.calculateDopplerShift(v_los_planet_hydro)
    NaD1_cm, NaD2_cm = 5.897558147e-5, 5.891583253e-5

    for i, R_spectrum in enumerate(R_values_hydro):
        shift_factor = shifts_hydro[i]
        shifted_D1 = NaD1_cm / shift_factor
        shifted_D2 = NaD2_cm / shift_factor
        sel1 = (transit_model_hydro.wavelength >= shifted_D1 - FILTER_BANDWIDTH_CM / 2) & \
               (transit_model_hydro.wavelength <=
                shifted_D1 + FILTER_BANDWIDTH_CM / 2)
        sel2 = (transit_model_hydro.wavelength >= shifted_D2 - FILTER_BANDWIDTH_CM / 2) & \
               (transit_model_hydro.wavelength <=
                shifted_D2 + FILTER_BANDWIDTH_CM / 2)

        if np.any(sel1 | sel2):
            light_curve_sim_hydro.append(np.mean(R_spectrum[sel1 | sel2]))
        else:
            light_curve_sim_hydro.append(1.0)
else:
    print("Simulation did not produce a valid light curve array.")

Transmission spectra generation

In [ ]:
OBSERVED_DATA_FILEPATH = input(
    "Enter path to observed spectrum data: ").strip()
obs_wavelength, obs_transmission, obs_errors = load_observed(
    OBSERVED_DATA_FILEPATH)
if obs_wavelength is None:
    print("Could not load observed spectrum data. Continuing without it.")

In [ ]:
params = get_plume_params(
    PLANET_NAME, ELEMENT_NAME, NUM_PARTICLES, SIGMA_V, PLUME_RADIUS,
    plume_longitude, MOON_EXOSPHERE_Q,
    light_curve=False, observed_planet_phases=obs_phases,
    observed_wavelength_transmissions=obs_wavelength
)

fundamentalsDict = params["Fundamentals"]
scenarioDict = params["Scenarios"]
architectureDict = params["Architecture"]
speciesDict = params["Species"]
gridsDict = params["Grids"]

planet = bodies.AvailablePlanets().findPlanet(
    architectureDict["planetName"])
if fundamentalsDict["CLV_variations"]:
    planet.hostStar.addCLVparameters(
        architectureDict.get("CLV_u1", 0.0), architectureDict.get("CLV_u2", 0.0))

wavelengthGrid = gasprop.WavelengthGrid(
    gridsDict["lower_w"], gridsDict["upper_w"], gridsDict["widthHighRes"],
    gridsDict["resolutionLow"], gridsDict["resolutionHigh"])

spatialGrid = geom.Grid(
    gridsDict["x_midpoint"], gridsDict["x_border"], int(
        gridsDict["x_steps"]),
    gridsDict["upper_rho"], int(
        gridsDict["rho_steps"]), int(gridsDict["phi_steps"]),
    gridsDict["orbphase_border"], int(gridsDict["orbphase_steps"]))

In [ ]:
scenarioList = []
if "exomoon" in scenarioDict:
    moon = bodies.Moon(
        architectureDict["starting_orbphase_moon"],
        architectureDict["R_moon"], architectureDict["a_moon"], planet)
    species_name = list(speciesDict["exomoon"].keys())[0]
    num_p = speciesDict["exomoon"][species_name]["Nparticles"]
    scenarioList.append(gasprop.MoonExosphere(
        num_p, scenarioDict["exomoon"]["q_moon"], moon))

In [ ]:
for idx, scenario_key in enumerate(scenarioDict.keys()):
    for species_key, absorberDict in speciesDict[scenario_key].items():
        if species_key in const.AvailableSpecies().listSpeciesNames():
            if "sigma_v" in absorberDict:
                scenarioList[idx].addConstituent(
                    species_key, absorberDict["sigma_v"])
                scenarioList[idx].constituents[-1].addLookupFunctionToConstituent(
                    wavelengthGrid)
            else:
                raise ValueError(
                    f"Missing 'sigma_v' for atomic species {species_key}")
        else:  # Molecular
            raise NotImplementedError(
                "Molecular species not fully supported in this script version.")

In [ ]:
atmos = gasprop.Atmosphere(
        scenarioList, fundamentalsDict["DopplerOrbitalMotion"])
main_transit_model = gasprop.Transit(atmos, wavelengthGrid, spatialGrid)
main_transit_model.addWavelength()

print("\nRunning PROMETHEUS simulation...")
start_time = datetime.now()
R_values_all_wavelengths = main_transit_model.sumOverChords()
end_time = datetime.now()
print(
    f"PROMETHEUS simulation finished. Elapsed time: {end_time - start_time}")

In [ ]:
if R_values_all_wavelengths.ndim == 2 and R_values_all_wavelengths.shape[0] > 0:
    spectrum_to_plot = R_values_all_wavelengths[0, :].copy()
    continuum_level = np.max(spectrum_to_plot)
    spectrum_to_plot /= continuum_level

    plot_transmission_spectrum(
        main_transit_model.wavelength, spectrum_to_plot,
        ELEMENT_NAME, PLANET_NAME, f"{NUM_PARTICLES:.1e}", f"{SIGMA_V:.1e}",
        obs_wavelength, obs_transmission, obs_errors)
else:
    print("Simulation did not produce a valid spectrum array.")

# Print max depth of the transmission spectrum
if R_values_all_wavelengths.ndim == 2 and R_values_all_wavelengths.shape[0] > 0:
    max_depth = np.min(R_values_all_wavelengths[0, :]) - 1.0
    print(f"Maximum depth of the transmission spectrum: {max_depth:.4f} (relative to continuum)")

In [ ]:
# Generate and plot the transmission spectrum for the hydrostatic scenario
hydrostatic_params_spec = get_hydrostatic_params(
    planet_name=PLANET_NAME,
    element_name=ELEMENT_NAME,
    mixing_ratio=mix_ratio_hydro,
    temperature=atmos_temp_hydro,
    reference_pressure_bar=ref_pressure_hydro,
    mean_molecular_weight_amu=mean_mu_hydro,
    light_curve=False
)

fundamentalsDict_hydro_spec = hydrostatic_params_spec["Fundamentals"]
scenarioDict_hydro_spec = hydrostatic_params_spec["Scenarios"]
architectureDict_hydro_spec = hydrostatic_params_spec["Architecture"]
speciesDict_hydro_spec = hydrostatic_params_spec["Species"]
gridsDict_hydro_spec = hydrostatic_params_spec["Grids"]

planet_hydro_spec = bodies.AvailablePlanets().findPlanet(
    architectureDict_hydro_spec["planetName"])
if fundamentalsDict_hydro_spec["CLV_variations"]:
    planet_hydro_spec.hostStar.addCLVparameters(
        architectureDict_hydro_spec.get("CLV_u1", 0.0), architectureDict_hydro_spec.get("CLV_u2", 0.0))

wavelengthGrid_hydro_spec = gasprop.WavelengthGrid(
    gridsDict_hydro_spec["lower_w"], gridsDict_hydro_spec["upper_w"], gridsDict_hydro_spec["widthHighRes"],
    gridsDict_hydro_spec["resolutionLow"], gridsDict_hydro_spec["resolutionHigh"])

spatialGrid_hydro_spec = geom.Grid(
    gridsDict_hydro_spec["x_midpoint"], gridsDict_hydro_spec["x_border"], int(gridsDict_hydro_spec["x_steps"]),
    gridsDict_hydro_spec["upper_rho"], int(gridsDict_hydro_spec["rho_steps"]), int(gridsDict_hydro_spec["phi_steps"]),
    gridsDict_hydro_spec["orbphase_border"], int(gridsDict_hydro_spec["orbphase_steps"]))

scenario_model_hydro_spec = gasprop.HydrostaticAtmosphere(
    hydrostatic_params_spec["Scenarios"]["hydrostatic"]["T"],
    hydrostatic_params_spec["Scenarios"]["hydrostatic"]["P_0"],
    hydrostatic_params_spec["Scenarios"]["hydrostatic"]["mu"],
    planet_hydro_spec
)
scenario_model_hydro_spec.addConstituent(ELEMENT_NAME, hydrostatic_params_spec["Species"]["hydrostatic"][ELEMENT_NAME]["chi"])
scenario_model_hydro_spec.constituents[-1].addLookupFunctionToConstituent(wavelengthGrid_hydro_spec)

atmos_hydro_spec = gasprop.Atmosphere([scenario_model_hydro_spec], fundamentalsDict_hydro_spec["DopplerOrbitalMotion"])
transit_model_hydro_spec = gasprop.Transit(atmos_hydro_spec, wavelengthGrid_hydro_spec, spatialGrid_hydro_spec)
transit_model_hydro_spec.addWavelength()

print("\n--- Running PROMETHEUS for Hydrostatic Transmission Spectrum... ---")
R_values_hydro_spec = transit_model_hydro_spec.sumOverChords()
print("Hydrostatic transmission spectrum simulation finished.")

if R_values_hydro_spec.ndim == 2 and R_values_hydro_spec.shape[0] > 0:
    spectrum_to_plot_hydro = R_values_hydro_spec[0, :].copy()
    continuum_level_hydro = np.max(spectrum_to_plot_hydro)
    spectrum_to_plot_hydro /= continuum_level_hydro

    plot_transmission_spectrum(
        transit_model_hydro_spec.wavelength, spectrum_to_plot_hydro,
        ELEMENT_NAME, PLANET_NAME, f"Mix Ratio: {mix_ratio_hydro:.1e}", f"Temp: {atmos_temp_hydro}K",
        obs_wavelength, obs_transmission, obs_errors)
else:
    print("Simulation did not produce a valid spectrum array for the hydrostatic case.")
    
# Print max depth for hydrostatic atmosphere
if R_values_hydro_spec.ndim == 2 and R_values_hydro_spec.shape[0] > 0:
    max_depth_hydro = np.min(R_values_hydro_spec[0, :]) - 1.0
    print(f"Maximum absorption depth for hydrostatic atmosphere: {max_depth_hydro:.4f} (relative to continuum)")